In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit


--2020-07-09 22:00:44--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-07-09 22:00:44--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  84.75M   196MB/s    in 0.4s    

2020-07-09 22:00:45 (196 MB/s) - ‘Miniconda3-latest-Linux

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import numpy as np
import pandas as pd

from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps

import glob
import imageio
from tqdm import tqdm

import matplotlib.pyplot as plt
# matplotlib.use('agg')

import torch

import os

In [ ]:
# We first process the data into tuples: the first element is the torch tensor
# for the image, and the second is the hydrophobicity/hydrophilicity label. 
# Note that the solubility units are given in lopS (log of molar solubility)

from google.colab import drive
drive.mount('/content/gdrive')   #mount at GFS
dir_path = '/content/gdrive/My Drive/Colab Notebooks/Msc Project/image'
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Msc Project/solubility-dataset.csv') # <--- replace this with your data directory
med = np.median(data['Solubility'])
# If hydrophobic, then label 1, else label 0
data['Solubility'] = np.where(data['Solubility']<=med, 0, 1)
# If hydrophobic, then label 1, else label0

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# Some general drawing parameters. The idea is to draw the molecules
# without any atomic labels and just focusing on partial charge density
# and the color of the bonds (because the atoms are colored). I would expect
# that molecules with high partial charge densities are soluble in water and 
# low partial charge densities are not soluble in water.

n_mols = data.shape[0]
Draw.DrawingOptions.bondLineWidth = 2
Draw.DrawingOptions.atomLabelFontSize = 1
Draw.DrawingOptions.atomLabelMinFontSize = 1
trim_margin = 20


In [ ]:
def shapeParameters():
    """
    This function calculates the appropriate shape of im_tensor so that we
    can appropriately concat all image matrices to it. We need to be able to 
    preassign the shape correctly
    """

    # margin and other parameters. I first generate and image and get the
    # parameters from there. The only user parameter required is the trim
    # width.
    i = 10
    smiles = data['SMILES'][i]
    category = data['Solubility'][i]

    mol = Chem.MolFromSmiles(smiles)
    Chem.ComputeGasteigerCharges(mol)
    contribs = ([float(mol.GetAtomWithIdx(i).GetProp('_GasteigerCharge')) 
                for i in range(mol.GetNumAtoms())])
    filename = dir_path + '/figs/mol' + str(i) + '.png'
    fig = SimilarityMaps.GetSimilarityMapFromWeights(mol, contribs,scale=150,
                                     colorMap='bwr', 
                                     contourLines=1,
                                     size=(250, 250))
    fig.savefig(filename, bbox_inches='tight')
    
    im = imageio.imread(filename)
    height, width, channels = im.shape
    trimmed_height, trimmed_width = (height - 2*trim_margin, 
                                             width - 2*trim_margin)
    
    return trimmed_width, trimmed_height


In [ ]:
# Note that instead of saving and then reloading, we could have just done it
# by directly converting the image into the matrix. But there is something about 
# rdkit not fitting the figure inside the canvas.

# I will have to do the figure saving and the generation together because
# some images are being skipped and then the categories wil not match properly
#%%
def molsToImgsToTensor():
    """
    This function takes all the molecules in the dataset, generates images of
    these molecules and daves these as png images
    """
    # We need to preallocate the tensor in memory because append/concat is very
    # slow. We will have a bunch of elements at the end which we will delete
    # before returning the result by maintaining a counter.
    # %matplotlib agg
    trimmed_height, trimmed_width = shapeParameters()
    n_mols = data.shape[0]  
    im_tensor = torch.zeros((n_mols, trimmed_height, trimmed_width, 4),
                            dtype=torch.uint8)
    category_tensor = torch.zeros((n_mols, 1))
    counter = 0
    for i in tqdm(range(data.shape[0])):
        try:
            smiles = data['SMILES'][i]
            category = torch.Tensor([data['Solubility'][i]]).view(1,-1)
            mol = Chem.MolFromSmiles(smiles)
            Chem.ComputeGasteigerCharges(mol)
            contribs = ([float(mol.GetAtomWithIdx(i).GetProp('_GasteigerCharge')) 
                        for i in range(mol.GetNumAtoms())])
            filename = dir_path + '/figs/mol' + str(i) + '.png'
            fig = SimilarityMaps.GetSimilarityMapFromWeights(mol, contribs, scale=150,
                                              colorMap='bwr', 
                                              contourLines=1,
                                              size=(250, 250))
            fig.savefig(filename, bbox_inches='tight')
            
            im = imageio.imread(filename)
            height, width, channels = im.shape
            
            im = im[trim_margin:-trim_margin, trim_margin:-trim_margin, :]
            im = torch.from_numpy(im).view(1, trimmed_width, trimmed_height, 4)
            im_tensor[counter] = im
            # im_tensor = torch.cat((im_tensor, im), dim=0)
            category_tensor[counter] = category
            # category_tensor = torch.cat((category_tensor, category),
                                        # dim=0)
            counter += 1
        except:
            pass
    return (counter, im_tensor.numpy()[:counter], 
                     category_tensor.int().numpy()[:counter])

In [ ]:
counter, im_tensor, category_tensor = molsToImgsToTensor()

np.save('/content/gdrive/My Drive/Colab Notebooks/Msc Project/image/im_tensor', im_tensor)
np.save('/content/gdrive/My Drive/Colab Notebooks/Msc Project/image/category_tensor', category_tensor)

  0%|          | 19/9982 [00:08<1:16:02,  2.18it/s]/usr/local/lib/python3.7/site-packages/rdkit/Chem/Draw/mplCanvas.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = figure(figsize=figsize)
  1%|          | 89/9982 [00:41<1:14:16,  2.22it/s]/usr/local/lib/python3.6/dist-packages/matplotlib/contour.py:1483: UserWarning: Warning: converting a masked element to nan.
  self.zmax = float(z.max())
/usr/local/lib/python3.6/dist-packages/matplotlib/contour.py:1484: UserWarning: Warning: converting a masked element to nan.
  self.zmin = float(z.min())
/usr/local/lib/python3.6/dist-packages/matplotlib/contour.py:1132: RuntimeWarning: invalid value encountered in less
  under = np.nonzero(lev < self.zmin)[0]
/usr/local/lib/python3.6/dist-packages/matplotlib/

In [ ]:
# I still need to remove the frame while converting the figures into 
# torch tensors. For this, I can just import the figures and then trim the edges
# by taking image slices. I will have to go in and delete images that look weird

# Note that there are tons of while pixels, but for now I am just going to
# have to do the convolution with it. Maybe this is also a good chance  to 
# see how well the neural network is able to pick out the features when there
# is such a large set of white pixels. I also noticed that there are some
# images that are fully black. I am also going to leave these in to add 'noise'
# to my dataset to see how robust my code is and how good it is at capturing
# stuff.


# At this point we have a tensor that contains all the images. We then take this
# tensor and convert this into a data training pair in the main file
